# Aplica variantes del algoritmo k-medias y otros métodos particionales 
En este notebook aprenderás a ejecutar varios métodos de agrupamiento particional, como alternativa al uso de k-medias. Seguiremos trabajando con las dos librerías Python utilizadas la semana anterior, *PyClustering* y *scikit-learn*, ya que cada una de ellas tiene implementaciones de métodos diferentes.
## 1. Cambiar la inicialización de k-medias en ***PyClustering***
En la primera parte del *notebook* utilizaremos la librería *PyClustering*, cuya [documentación en línea](https://pyclustering.github.io/docs/0.10.1/html/index.html) nos ayudará a configurar y ejecutar variantes del método k-medias. En esta librería, cada método de agrupamiento está definido en su propio subpaquete dentro de *cluster*. En este primer apartado, solo necesitamos importar un nuevo tipo de inicialización denominada *k-means++*.

In [ ]:
# La primera vez que se vaya a ejecutar este notebook es necesario instalar la librería pyclustering
!pip install pyclustering
from pyclustering.cluster.kmeans import kmeans, kmeans_visualizer
from pyclustering.cluster.center_initializer import random_center_initializer, kmeans_plusplus_initializer
import numpy as np
import matplotlib.pyplot as plt

Vamos a trabajar con un conjunto de datos precargado en *PyClustering*, donde los grupos están bastante dispersos. Para ver su distribución, podemos utilizar un visor proporcionado por la propia librería.

In [ ]:
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import SIMPLE_SAMPLES
from pyclustering.cluster import cluster_visualizer
datos = read_sample(SIMPLE_SAMPLES.SAMPLE_SIMPLE3)
visor = cluster_visualizer()
visor.append_cluster(datos, color="blue")
grafico_datos = visor.show()


La semana anterior vimos que antes de ejecutar el algoritmo k-medias en *PyClustering* necesitamos generar los centroides iniciales. Aparte de la inicialización aleatoria, la librería nos da la opción de utilizar una variante de k-medias denominada [*k-means++*](https://pyclustering.github.io/docs/0.10.1/html/db/de0/classpyclustering_1_1cluster_1_1center__initializer_1_1kmeans__plusplus__initializer.html#details). Este método proporciona una forma mejorada para determinar los centroides antes de ejecutar *k-means* u otro algoritmo de su misma "familia".

In [ ]:
k = 4
centroides_iniciales_optimizados = kmeans_plusplus_initializer(datos, k, random_state=0).initialize()
print(centroides_iniciales_optimizados)

Además de asignar los centroides obtenidos por este nuevo método, vamos a limitar el número de iteraciones del algoritmo a 5 con el parámetro *itermax*. De esta forma, podremos apreciar más claramente la influencia de los centroides iniciales, ya que el algoritmo tendrá pocas iteraciones para converger hacia los mejores centroides.

In [ ]:
alg_kmeans_optimizado = kmeans(datos, centroides_iniciales_optimizados, itermax=5)

Podemos representar gráficamente la ubicación de estos centroides para ver que se ajustan con bastante precisión a los grupos que se observan en el conjunto de datos, ya que esta inicialización pre-analiza la distancia entre puntos.

In [ ]:
visor.append_cluster(centroides_iniciales_optimizados, marker='*', markersize=10, color='green')
grafico_datos = visor.show()

Vamos a inicializar también el conjunto de centroides de manera aleatoria para comparar cómo afecta al análisis de grupos con k-medias. Utilizamos el parámetro *random_state* para fijar una semilla aleatoria. Con ello, aseguramos que cualquier nueva ejecución de la función nos devolverá la misma inicialización de centroides.

In [ ]:
centroides_iniciales_aleatorios = random_center_initializer(datos, k, random_state=0).initialize()
print(centroides_iniciales_aleatorios)
alg_kmeans_aleatorio = kmeans(datos, centroides_iniciales_aleatorios, itermax=5)


Comprobamos con el visor cuál es la ubicación de estos centroides aleatorios.

In [ ]:
visor = cluster_visualizer()
visor.append_cluster(datos, color="blue")
visor.append_cluster(centroides_iniciales_aleatorios, marker='X', markersize=10, color="red")
grafico_datos = visor.show()

A continuación, ejecutamos el análisis de grupos para cada instancia del algoritmo k-medias.

In [ ]:
alg_kmeans_optimizado.process()
alg_kmeans_aleatorio.process()

Para ver el resultado, utilizamos el visor de grupos disponible en el propio algoritmo k-medias, que nos muestra de forma conjunta los grupos y los centroides. A pesar de realizar pocas iteraciones, el resultado agrupa de forma correcta las instancias del conjunto de datos.

In [ ]:
grafico_optimizado = kmeans_visualizer.show_clusters(datos, alg_kmeans_optimizado.get_clusters(), alg_kmeans_optimizado.get_centers())

Si hacemos lo mismo con el algoritmo cuyos centroides iniciales eran aleatorios, vemos claramente que no es capaz de encontrar una distribución tan adecuada como en el caso anterior.

In [ ]:
grafico_aleatorio = kmeans_visualizer.show_clusters(datos, alg_kmeans_aleatorio.get_clusters(), alg_kmeans_aleatorio.get_centers())

## 2. Utilizar variantes de k-medias en ***PyClustering***
Hemos visto como cambiar uno de los elementos que afectan al algoritmo k-medias, su inicialización. En este apartado vamos a ver otros algoritmos que son variantes de k-medias puesto que modifican la forma en la que se determinan los centroides. Estos otros algoritmos son *k-medians* y *k-medoids*.

In [ ]:
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster.kmedoids import kmedoids

En primer lugar, vamos a generar un conjunto de datos de dos variables y 50 puntos generados de forma aleatoria.

In [ ]:
tam = 50
datos = np.random.random((tam,2))
x = datos[0:tam,0]
y = datos[0:tam,1]
plt.scatter(x, y)
plt.show()


Para k-medias, podemos realizar la inicialización optimizada como en el apartado anterior. A continuación, ejecutamos el algoritmo y visualizamos los grupos. Vamos a utilizar un semilla aleatoria para poder replicar esta inicialización más adelante.

In [ ]:
k = 5
centroides_iniciales = random_center_initializer(datos,k,random_state=0).initialize()
alg_kmeans = kmeans(datos, centroides_iniciales)
alg_kmeans.process()
grafico = kmeans_visualizer.show_clusters(datos, alg_kmeans.get_clusters(), alg_kmeans.get_centers())

Para k-medians, podemos utilizar los mismos centroides iniciales para comprobar que el cambio en la asignación de grupos se debe al re-cálculo de centroides. En este algoritmo, el centroide se calcula como la mediana en lugar de la media.

In [ ]:
medians_iniciales = centroides_iniciales
alg_kmedians = kmedians(datos, medians_iniciales)
alg_kmedians.process()
grafico = kmeans_visualizer.show_clusters(datos, alg_kmedians.get_clusters(), alg_kmedians.get_medians())

Para k-medoids, los centroides van a corresponderse con puntos reales del conjunto de datos.En primer lugar, tenemos que indicarle al generador de centroides iniciales que devuelva los índices de los puntos elegidos en lugar de sus coordenadas.

In [ ]:
medoids_iniciales = random_center_initializer(datos,k,random_state=0).initialize(return_index=True)
alg_kmedoids = kmedoids(datos, medoids_iniciales)
alg_kmedoids.process()

visor = cluster_visualizer()
visor.append_clusters(alg_kmedoids.get_clusters(), datos)
visor.append_cluster(alg_kmedoids.get_medoids(), datos, markersize=14, marker='*', color='black')
grafico = visor.show()

Los centroides finales de *k-medoids* son índices del conjunto de datos, aquellos elegidos como representantes de cada grupo. Por el contrario, en k-medias y *k-medians* se calculan en base a la asignación de grupos, por lo que se devuelven en forma de coordenadas. Lo más habitual es que no coincidan con puntos reales del conjunto de datos, aunque en el caso de *k-medians* es más posible que alguno sí lo sea. Podemos comprobarlo con el sigeuiente código.

In [ ]:
# Centroides en k-means
print('Centroides en k-medias: punto, ¿está en el conjunto de datos?')
centroides_kmeans = alg_kmeans.get_centers()
for i in range(0, len(centroides_kmeans)):
  c = centroides_kmeans[i]
  print(c, c in datos)

# Centroides en k-medians
print('Centroides en k-medians: punto, ¿está en el conjunto de datos?')
centroides_kmedians = alg_kmedians.get_medians()
for i in range(0, len(centroides_kmedians)):
  c = centroides_kmedians[i]
  print(c, c in datos)

# Centroides en k-medoids
print('Centroides en k-medias: índice, punto')
centroides_kmedoids = alg_kmedoids.get_medoids()
for i in range(0, len(centroides_kmedoids)):
  c = centroides_kmedoids[i]
  print(c, datos[c])

## 3. Cambiar la inicialización de k-medias en ***scikit-learn***
Al igual que en *PyClustering*, es posible cambiar el método de inicialización para el algoritmo k-medias en *scikit-learn*. En este apartado veremos cómo hacerlo. Lo primero que necesitamos es importar el algoritmo.

In [ ]:
from sklearn.cluster import KMeans

A continuación, creamos nuestro conjunto de datos.

In [ ]:
tam = 100
datos = np.random.random((tam,2))
x = datos[0:tam,0]
y = datos[0:tam,1]
plt.scatter(x, y)
plt.show()

Lo siguiente que tenemos que hacer es crear la instancia del algoritmo k-medias. Si consultamos la [documentación de referencia](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans), Uno de sus parámetros es *init*. Este parámetro puede tomar dos valores: *'random** y *'k-means++'*. Actualmente, el valor por defecto es *k-means++*, pues se ha visto que funciona mejor que el método aleatorio. No obstante, podemos indicarlo expresamente en la llamada la función. Además, vamos a reducir el número máximo de iteraciones (parámetro *max_iter*) para ver más claramente el beneficio de esta inicialización frente a la aleatoria.

In [ ]:
k = 4
alg_kmeans_plusplus = KMeans(n_clusters=k, random_state=1, init='k-means++', max_iter=5)
alg_kmeans_plusplus.fit(datos)

Hacemos el mismo proceso, pero indicando que la inicialización sea aleatoria.

In [ ]:
alg_kmeans = KMeans(n_clusters=k, random_state=1, init='random', max_iter=5)
alg_kmeans.fit(datos)

Vamos a definir una función que nos permita visualizar una agrupación determinada, de forma que podamos comparar la asignación de cada algoritmo.

In [ ]:
def visualizar_grupos_sklearn(x, y, etiquetas, centroides, k):
  plt.scatter(x, y, c=etiquetas)
  colores = np.arange(0, k)
  plt.scatter(centroides[0:k,0], centroides[0:k,1], marker="*", c=colores)
  plt.show()


Ya podemos visualizar los resultados de k-medias y de k-medias++.

In [ ]:
visualizar_grupos_sklearn(x, y, alg_kmeans_plusplus.labels_, alg_kmeans_plusplus.cluster_centers_, k)

In [ ]:
visualizar_grupos_sklearn(x, y, alg_kmeans.labels_, alg_kmeans.cluster_centers_, k)

## 4. Ejecutar AffinityPropagation en ***scikit-learn***
[AffinityPropagation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html#sklearn.cluster.AffinityPropagation) es otro algoritmo particional que está disponible en la librería *scikit-learn*. Este algoritmo, a diferencia de *k-medias*, no require indicar el número de grupos a descubrir (parámetro *k*). 

In [ ]:
from sklearn.cluster import AffinityPropagation

Para ejecutarlo, seguimos el mismo procedimiento de configuración que con otros algoritmos de esta librería.

In [ ]:
alg_aff_prop = AffinityPropagation()
alg_aff_prop.fit(datos)

Para saber cuántos grupos ha identificado, podemos recuperar los el vector de centroides (*exemplars*) y ver su dimensión. Además, como estos centroides son puntos reales del conjunto de datos, podemos recuperar también el índice de la instancia del conjunto de datos con el cual se corresponde cada uno.

In [ ]:
exemplars = alg_aff_prop.cluster_centers_
num_grupos = len(exemplars)
print(num_grupos)
indices = alg_aff_prop.cluster_centers_indices_
print(indices)

 Por úlitmo, probamos a visualizar la partición devuelta por este algoritmo con nuestra función de visualización.

In [ ]:
visualizar_grupos_sklearn(x, y, alg_aff_prop.labels_, exemplars, num_grupos)